# 🎨 Data Designer Tutorial: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    ChatCompletionInferenceParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    LocalFileSeedSource,
    ModelConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) are used.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v3"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=2048,
            extra_body={"chat_template_kwargs": {"enable_thinking": False}},
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, _ = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

# Seed datasets are passed as reference objects to the config builder.
seed_source = LocalFileSeedSource(path=local_filename)

config_builder.with_seed_dataset(seed_source)

DataDesignerConfigBuilder()

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
Respond with only the notes, no other text.
""",
    model_alias=MODEL_ALIAS,
)

data_designer.validate(config_builder)

[21:54:28] [INFO] ✅ Validation passed


### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer.preview(config_builder, num_records=2)

[21:54:28] [INFO] 🔁 Preview generation in progress


[21:54:28] [INFO] ✅ Validation passed


[21:54:28] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[21:54:28] [INFO] 🩺 Running health checks for models...


[21:54:28] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[21:54:34] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[21:54:34] [INFO]   |-- 🎲 column: 'date_of_visit'


[21:54:34] [INFO]   |-- 🧩 column: 'physician'


[21:54:34] [INFO]   |-- 📝 column: 'physician_notes'


[21:54:34] [INFO] 👏 Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': '56aa27d8-c859-40d4-bea8-54314990bd7c',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Carly',                                                                 │
│                    │     'last_name': 'Cooper',                                                                 │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Female',                                                                       │
│                    │     'street_number': '48657',                                                              │
│                    │     'street_name': 'Steele Mission',                                                       │
│                    │     'city': 'South Jorgeton',                                                              │
│                    │     'state': 'Connecticut',                                                                │
│                    │     'postcode': '96386',                                                                   │
│                    │     'age': 58,                                                                             │
│                    │     'birth_date': '1967-08-25',                                                            │
│                    │     'country': 'Sierra Leone',                                                             │
│                    │     'marital_status': 'never_married',                                                     │
│                    │     'education_level': 'graduate',                                                         │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Child psychotherapist',                                                 │
│                    │     'phone_number': '2515908672',

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': '56aa27d8-c859-40d4-bea8-54314990bd7c...,{'uuid': '3bbe30a9-3925-4219-94de-ed8e57d5d72d...,PT-A754975F,2024-05-19,2024-06-13,Carly,Cooper,1967-08-25,Dr. Powell,**Patient:** Carly Cooper \n**DOB:** [Insert ...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': '0a150989-90ed-4d81-bab9-fd9febe71b1e...,{'uuid': 'd1e864ec-72c1-42ef-b4f8-926dd67c90db...,PT-C41625B3,2024-02-17,2024-02-28,Christine,Thompson,1972-06-09,Dr. Phillips,**Visit: 2024-02-28 | Patient: Christine Thomp...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                               │ 10                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       2 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       2 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       2 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ physician_notes       │        string │                 2 (100.0%) │     122.0 +/- 4.0 │       1132.0 +/- 172.5 │
└───────────────────────┴───────────────┴────────────────────────────┴───────────────────┴────────────────────────┘
                                                                                                                   
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
results = data_designer.create(config_builder, num_records=10, dataset_name="tutorial-3")

[21:54:34] [INFO] 🎨 Creating Data Designer dataset


[21:54:34] [INFO] ✅ Validation passed


[21:54:34] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[21:54:34] [INFO] 🩺 Running health checks for models...


[21:54:34] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[21:54:49] [INFO]   |-- 🧩 column: 'dob'


[21:54:49] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[21:54:49] [INFO]   |-- 🎲 column: 'date_of_visit'


[21:54:49] [INFO]   |-- 🧩 column: 'physician'


[21:54:49] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 89, 'bachelors_field': 'no_degree', 'b...","{'age': 93, 'bachelors_field': 'stem_related',...",PT-D7A046C9,2024-05-23,2024-06-06,Donald,Bowman,1936-03-30,Dr. Mason,**Patient:** Donald Bowman **DOB:** 04/12/19...
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 48, 'bachelors_field': 'no_degree', 'b...","{'age': 42, 'bachelors_field': 'no_degree', 'b...",PT-32E3F5F2,2024-04-02,2024-04-09,Jay,Keller,1977-09-24,Dr. Adams,**Visit Note – 2024-04-09** **Patient:** Jay...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 87, 'bachelors_field': 'stem_related',...","{'age': 106, 'bachelors_field': 'stem', 'birth...",PT-41DC7182,2024-12-12,2024-12-23,Brandon,Moore,1938-07-26,Dr. Velez,**Patient:** Brandon Moore **DOB:** 03/15/19...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 24, 'bachelors_field': 'no_degree', 'b...","{'age': 103, 'bachelors_field': 'no_degree', '...",PT-54FB351B,2024-01-19,2024-02-06,Amanda,Myers,2001-04-03,Dr. Pratt,**Visit Summary: Amanda Myers | 2024-02-06 | P...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 86, 'bachelors_field': 'stem', 'birth_...","{'age': 102, 'bachelors_field': 'no_degree', '...",PT-04820BC4,2024-03-03,2024-04-01,Darryl,Torres,1939-05-12,Dr. Holmes,**Patient:** Darryl Torres **DOB:** 1992-06-...


In [13]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 10                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      10 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ physician_notes       │        string │                10 (100.0%) │     118.5 +/- 5.5 │        929.5 +/- 198.8 │
└───────────────────────┴───────────────┴────────────────────────────┴───────────────────┴────────────────────────┘
                                                                                                                   
                                                          

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Providing images as context](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/4-providing-images-as-context/)
